# Searching for a minimum string cover

What is the shortest sequence of English words that contains each of the following as a substring? (These are the cities where [Futurice](https://futurice.com/) offices are located.) 

In [1]:
# must be on lower case
searchkeys = ["hel", "tre", "ber", "mun", "lon", "sto"];

To make this more interesting, the search keys can span over word boundaries. That is, the sequence "willful onion" covers the search key *lon* because the first word ends with *l* and the second word starts with *on*. The search keys are allowed to overlap, that is, the word "echelon" covers both *hel* and *lon* with the letter *l* shared between them.

This is an instance of [the set cover problem](https://en.wikipedia.org/wiki/Set_cover_problem) in computer science.

We can get a list of English words from `/usr/share/dict/words`.

In [2]:
function load_vocabulary(filename)
    map(lowercase,
        map(x -> replace(x, "'", ""),
        map(x -> replace(x, "-", ""),
        map(strip, readlines(filename)))))
end

vocabulary = load_vocabulary("/usr/share/dict/words")

println("$(length(vocabulary)) words in the vocabulary")

100411 words in the vocabulary


## The simplest thing that could possibly work

We could simply form every possible sequence of vocabulary words up to length of six words (at most six words are needed to cover the six search keys), check which of them contains all of the search keys, and keep track of the shortest one. However, that is way too slow because the number of candidates is exponential in the number of search keys. More precisely, the number of candidates equals $length(vocabulary)$ to the power of $length(searchkeys)$.

My laptop can evaluate about 10 million candidates in a second. Processing all candidates at that rate would take over 20 000 years. That is slightly longer than I'm willing to wait, so let's see if we can find a faster way.

## Greedy approximation

Even if we can't (yet) compute the optimal solution efficiently, we can approximate it easily. A greedy approximation groups the vocabulary words according to which search keys they cover, selects the shortest word for each search key, and concatenates the words. This is an approximation because it doesn't find (mostl likely shorter) cases where the search keys strecth over word boundaries. Nevertheless, the greedy approximation is reasonable good.

In [3]:
function cover_greedy(searchkeys, vocabulary)
    res = Vector{AbstractString}()
    for key in searchkeys
        shortest_for_key = sort(filter(w -> contains(w, key), vocabulary), by=length)[1]
        push!(res, shortest_for_key)
    end

    unique(res)
end

function pretty_search_result(terms)
    cost = sum(map(length, terms))    
    "$(join(terms, " ")), length: $cost"
end

greedy_res = cover_greedy(searchkeys, vocabulary)
print("Greedy search: ")
println(pretty_search_result(greedy_res))

Greedy search: hell trey berg munch lon stop, length: 24


## Preparing data structures for efficient search

Now that we have an approximation, let's focus again on finding the optimal answer.

One way to reduce the required computation is to realise that not all word in the vocabulary can appear in a minimal cover. For example, the word "clock" can be pruned because it doesn't cover any of the search keys even partially. We can filter out these kind of words. However, care must be taken to keep words that start or end with a partial search key match, such as "onion", which starts with a partial match of *lon*.

In [4]:
flat(A) = mapreduce(x -> isa(x,Array)? flat(x): x, vcat, [], A)

function filter_vocabulary(searchkeys, vocabulary)
    prefixes = flat(map(s -> [s[1:i] for i in 1:(endof(s)-1)], searchkeys))
    postfixes = flat(map(s -> [s[i:endof(s)] for i in 2:endof(s)], searchkeys))
    
    function covers_key(s::AbstractString)
        full = any(x -> contains(s, x), searchkeys)
        ends = any(x -> endswith(s, x), prefixes)
        starts = any(x -> startswith(s, x), postfixes)
        full || ends || starts
    end

    unique(filter(covers_key, vocabulary))
end

filtered_vocabulary = filter_vocabulary(searchkeys, vocabulary)
println("$(length(filtered_vocabulary)) words left after filtering")

52897 words left after filtering


Another insight is that we only have to cover each key once similar to the greedy approximation. If at any point during the search we know that we have already covered the search key *hel* we can skip all words in the vocabulary that cover *hel* (but only if they don't also cover some other search key!).

The function *build_continuations* below builds a dictionary of vocabulry words grouped by the search keys they cover.

Search keys crossing word boundaries need special attention. Algorithm below benefits on a fast way to access words that start with a partial search key. So, words starting with *l*, say, are collected in a vector because they can cover *hel* when concatenated with a word that ends with *he*. The algorithm makes an assumption that a search key is covered by at most two consecutive words. The assumption holds for the search keys in this notebook, but is not generally true! A three letter search key with the article *a* in the middle position could be covered by three words. For example, a search key *man* is be covered by the sequence "warm a nun". Longer search keys are more likely to violate the assumption. The algorithm still finds a short sequence, but it is no longer guaranteed to be a minimal one.

In [5]:
function build_continuations(
        searchkeys::Vector{ASCIIString},
        vocabulary::Vector{AbstractString})
    by_key = Dict(map(k -> k => Dict{AbstractString, Vector{AbstractString}}(), searchkeys))
    for key in searchkeys
        by_key[key][""] = sort(filter(w -> contains(w, key), vocabulary), by=length)
        
        boundaries = map(i -> (key[1:i], key[(i+1):end]), 1:length(key)-1)
        for boundary in boundaries
            by_key[key][boundary[1]] = sort(filter(w -> startswith(w, boundary[2]), vocabulary), by=length)
        end
    end

    by_key
end

continuations = build_continuations(searchkeys, filtered_vocabulary)

for key in searchkeys
    println("Words covering \"$(key)\": $(length(continuations[key][""]))")
    partial = filter(x -> x[1] != "", [(boundary, length(terms)) for (boundary, terms) in continuations[key]])
    counts = sort(partial, by=x -> length(x[1]))
    for (boundary, len) in counts
        println("  $boundary-boundary: $(len)")
    end
end

Words covering "hel": 244
  h-boundary: 339
  he-boundary: 2958
Words covering "tre": 387
  t-boundary: 2593
  tr-boundary: 3377
Words covering "ber": 654
  b-boundary: 153
  be-boundary: 4631
Words covering "mun": 107
  m-boundary: 1375
  mu-boundary: 1770
Words covering "lon": 242
  l-boundary: 64
  lo-boundary: 1770
Words covering "sto": 550
  s-boundary: 558
  st-boundary: 2022


## Depth first search

The search algorithm must evaluate all sequences of the vocabulary words. The sequences can be thought to form a tree. The inner branches of the tree are partial sequences, such as "hello nest" (covers *hel* and *lon*). Its child nodes are sequences that have one word concatenated to the end of the string, such as "hello nest otter" (covers *hel*, *lon*, and *sto*) and "hello nest osbert" (covers *hel*, *lon*, *sto*, *ber*) among others. The leaf nodes are sequences that cover all the search keys, such as "hello nest osbert remunerate".

[Depth first search](https://en.wikipedia.org/wiki/Depth-first_search) traverses a search tree as far as possible and then backtracks to explore another branches. While visiting the nodes, the algorithm can be made to keeps track of the shortest feasible sequence encountered so far. The algorithm will find the minimum because it will visit every node eventually.

Below is a straight-forward recursive implementation of the depth first search.

In [6]:
function depth_first_search_slow(continuations, searchkeys, prev_word)
    best_terms = []
    best_cost = 999999

    if isempty(searchkeys)
        return best_terms, 0
    else
        for key in searchkeys
            if haskey(continuations, key)
                for (prev_postfix, candidates) in continuations[key]
                    if endswith(prev_word, prev_postfix)
                        for next in candidates
                            next_cost = length(next)
                            filtered_keys = filter(k -> !contains(next, k) && k != key, searchkeys)
                            rest_terms, rest_cost = depth_first_search_slow(continuations, filtered_keys, next)
                            if next_cost + rest_cost < best_cost
                                best_cost = next_cost + rest_cost
                                best_terms = vcat([next], rest_terms)
                            end
                        end
                    end
                end
            end
        end
            
        return best_terms, best_cost
    end
end

function cover_dfs(searchkeys, vocabulary)
    continuations = build_continuations(searchkeys, vocabulary)
    depth_first_search_slow(continuations, searchkeys, "")[1]
end

dfs_res = cover_dfs(["hel", "lon", "mun"], filtered_vocabulary)
print("Slow depth-first search (with three search keys): ")
println(pretty_search_result(dfs_res))

Slow depth-first search (with three search keys): munch ell on, length: 10


Unfortunately, this is still too slow to be usable for more than three or four search keys. A rough estimate shows that the search with all six search keys would takes years. (But not thousands of years. We are making progress!)

## Further optimizations

### Pruning unnecessary branches

Depth first search keeps track of the best solution it has seen so far during the search. We can use that as an upper bound and prune whole branches off of the search tree when it is evident that they can not produce a shorter sequence than the current best solution. Concretely, if the best solution at some point of the search is 24 letters long and the current partial candidate has 20 letters, we can skip all words longer than 3 letters. The *build_continuations_with_keybits* function sorts the candidate lists by word length so that it is possible to skip the remaining of the list as soon as the candidate length grows above the limit.

We'll use the greedy solution from the above as in initial upper bound. That way we can already in the beginning of the search prune large parts of the tree.

This is not an approximation. Only branches that are guaranteed to not contain the optimal solution are skipped. This will still find the true optimum.

### Avoiding memory allocations

Profiling the *depth_first_search_slow* function revealed that it spends a considerable proportion of time allocating and releaseing memory. Below is an updated version of the algorithm where I have made several changes to avoid unnecessary memory allocations:
* A vector for holding the current evaluation candidate (a vector of words) is allocated once in the beginning and passed as reference to the search function (the *workspace* variable below).
* Similarly, the data structure for holding the best solution so far is allocated once (previously the best result was returend by each recursive function call requiring a vector construction on every function call).
* The data structure indicating which search keys are unprocessed is replaced by a bit vector (the variable *keybits*). A UInt32 bit vector takes less space than a Vector of Strings that was used before. Manipulating the bit vector is also much faster, particularly, if we precompute keys contained in each word as similar bit vectors. This is done in the *build_continuations_with_keybits* function below. The downside is that bit vectors are limited to at most 32 (or 64, if UInt64s are used) search keys.

These changes improve the performance considerably. I profiled each of these changes separately and a few other ideas that turned out not to improve the performance (the intermediate step are not shown here). However, each change also make the code more complicated. The tradeoff between performance and readibilty is often difficult.

In [7]:
# Returns a UInt32 bitset indicating which searchkeys are covered by word.
# For example, returns 101 if the first and third search key are covered
# by the word.
function word_to_keybits(word::AbstractString, searchkeys::Vector{ASCIIString})
    bits::UInt32 = 0
    b::UInt32 = 1
    
    if length(searchkeys) > 32
        error("Supports at most 32 search keys")
    end

    for key in searchkeys
        if contains(word, key)
            bits = bits | b
        end
        b = b << 1
    end
    
    bits
end

# This is like the build_continuations function above, but also include
# bitsets which indicate which search keys are covered by each word.
function build_continuations_with_keybits(
        searchkeys::Vector{ASCIIString},
        vocabulary::Vector{AbstractString})
    by_key = Dict(map(k -> k => Dict{AbstractString, Vector{Tuple{AbstractString, UInt32}}}(), searchkeys))
    
    for key in searchkeys
        words = sort(filter(w -> contains(w, key), vocabulary), by=length)
        by_key[key][""] = map(w -> (w, word_to_keybits(w, searchkeys)), words)

        boundaries = map(i -> (key[1:i], key[(i+1):end]), 1:length(key)-1)
        for boundary in boundaries
            words = sort(filter(w -> startswith(w, boundary[2]), vocabulary), by=length)
            by_key[key][boundary[1]] = map(w -> (w, word_to_keybits(boundary[1] * w, searchkeys)), words)
        end
    end

    by_key
end;

In [8]:
# A struct for representing the current best result
type Cover{T}
    terms::Vector{T}
    cost::Int
    level::Int
end

In [9]:
function depth_first_search_optimized!(
        searchkeys::Vector{ASCIIString},
        keybits::UInt32,
        continuations::Dict{ASCIIString, Dict{AbstractString, Vector{Tuple{AbstractString, UInt32}}}},
        level::Int,
        current_cost::Int,
        workspace::Vector{AbstractString},
        best::Cover{AbstractString})
    if keybits == 0
        # The current best solution is updated through a reference to
        # avoid the need to copy the vector on every function call.
        if current_cost < best.cost
            best.cost = current_cost
            best.terms[:] = workspace[:]
            best.level = level - 1
        end
    else
        if level > 1
            prev_word = workspace[level-1]
        else
            prev_word = ""
        end

        for i in 1:length(searchkeys)
            if keybits & (UInt32(1) << (i-1)) == 0
                continue
            end

            key = searchkeys[i]
            for (prev_postfix, candidates) in continuations[key]
                if endswith(prev_word, prev_postfix)
                    for (next, coverbits) in candidates
                        next_cost::Int = current_cost + length(next)
                        if next_cost > best.cost
                            # Every candidate beyond this is too long, 
                            # because the candidates are sorted by length
                            break
                        end
                        
                        # The active keys are updated with a bit operation
                        next_keybits = keybits & ~coverbits
                        workspace[level] = next
                        depth_first_search_optimized!(searchkeys, next_keybits, continuations, level + 1, next_cost, workspace, best)
                    end
                end
            end
        end
    end
    
    # Nothing to return here.
    # The actual return value is stored into the variable best.
    nothing
end

function all_keys_selected(searchkeys)
    bits = UInt32(0)
    for i in 1:length(searchkeys)
        bits = bits | (UInt32(1) << (i-1))
    end
    bits
end

function cover_dfs_optimized(searchkeys, vocabulary)
    greedy = cover_greedy(searchkeys, vocabulary)
    greedy_cost = sum(map(length, greedy))
    best = Cover{AbstractString}(greedy, greedy_cost, length(searchkeys))
    workspace = Vector{AbstractString}(length(searchkeys))
    keybits = all_keys_selected(searchkeys)
    continuations = build_continuations_with_keybits(searchkeys, vocabulary)
    depth_first_search_optimized!(searchkeys, keybits, continuations, 1, 0, workspace, best)
    best.terms[1:best.level]
end;

Let's run the search:

In [10]:
dfs_res_2 = cover_dfs_optimized(searchkeys, filtered_vocabulary)
print("Optimized depth first search: ")
println(pretty_search_result(dfs_res_2))

Optimized depth first search: hell on berm unit res to, length: 19


That took less than 5 minutes. Quite an improvement compared to the 20 000 years of the naive approach!

Notice how efficiently the search keys are being covered. There is only one letter in the solution, *i* in the word *unit*, that is not part of the search keys.

There might be other solutions with 19 letters. The code could be modified to visit all solutions that are the same length as the current best solution and store all of them in a list.

## Do you speak Finnish?

Now that we have the machinery ready, we can easily repeat the analysis with a different vocabulary. Let's find the shortest sequence of Finnish words containing the same substrings. I downloaded the list of Finnish words from [Kotus](http://kaino.kotus.fi/sanat/nykysuomi/).

In [11]:
vocabulary_fi = filter_vocabulary(searchkeys, load_vocabulary("kotus_sanat.txt"))
println("$(length(vocabulary_fi)) Finnish words left after filtering")

dfs_fi = cover_dfs_optimized(searchkeys, vocabulary_fi)
print("Optimized depth first search (Finnish words): ")
println(pretty_search_result(dfs_fi))

33531 Finnish words left after filtering
Optimized depth first search (Finnish words): helo nyt remu nosto weber, length: 21


The packing is slightly less efficent in Finnish as we have four letters that are not part of the search keys. On the other hand, now we got overlapping search keys *hel* and *lon*.

## Further (untested) optimization ides

* Recursive solution constantly re-allocates the stack. The function could be converted into a non-recursive one and the constant-sized space for stack could be allocated once. The upper limit for the required stack space is known because the recursion depth is at most the number of search keys.
* Maybe the code could be parallelized? The most important shared state is the best solution found so far.